In [1]:
import numpy as np
import polars as pl
import pandas as pd
from sklearn.base import clone
from copy import deepcopy
import optuna
from scipy.optimize import minimize
import os
import matplotlib.pyplot as plt
import seaborn as sns

import re
from colorama import Fore, Style
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from IPython.display import clear_output
from concurrent.futures import ThreadPoolExecutor

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

import lightgbm as lgb
from catboost import CatBoostRegressor, CatBoostClassifier
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import *
from sklearn.metrics import *

# Data Cleaning


<h2>Preparing the Data</h2>


<div style="font-size: 16px;">
Because some standard machine learning models cannot natively handle time series data, we have to transform time series data into a vector by extracting summary statistic to simplifies data representation and less computational complexity
</div>

In [2]:
def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet')) 
    # Read all parquet files that ends with part-0.parquet in series-train.parquet directory
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    # results is a tuple of statistic value with its id, like ([10.5, 20.3, 15.7], '001'),([12.1, 18.6, 14],'002')  
    stats, indexes = zip(*results)
    # now stats hold [10.5, 20.3, 15.7],... and indexes hold 001, 002,..
    df = pd.DataFrame(stats, columns=[f"Stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    
    return df

train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

100%|██████████| 2/2 [00:00<00:00,  9.52it/s]


In [3]:
time_series_cols = train_ts.columns.tolist()
time_series_cols.remove("id")

In [4]:
train = pd.merge(train, train_ts, how="left", on='id')
test = pd.merge(test, test_ts, how="left", on='id')
#We merge because train and train_ts, both of them have rows that correspond to a single child (with id)

In [5]:
# After merging we drop it cause we no longer need it
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

<div style="font-size: 16px;">
Dropping all the PCIAT-related columns because they are not in the test set
</div>

In [6]:
featuresCols = [
                'Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Season',
                'Fitness_Endurance-Max_Stage','Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 
                'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 
                'PAQ_C-Season',
                'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
                'PreInt_EduHx-computerinternet_hoursday', 'sii']

In [7]:
featuresCols += time_series_cols

In [8]:
train = train[featuresCols]

<h2>Handling missing values</h2>

<div style="font-size: 16px;">
We are dropping features that have more than 80% missing values
</div>

In [9]:
threshold = 0.8
target = train['sii']
train = train[[col for col in train.columns if col != 'sii']].dropna(axis=1, thresh=(1 - threshold) * len(train))
test = test[train.columns] 
train['sii'] = target

<h2>Removing Outliers</h2>

<div style="font-size: 16px;">
Replacing outliers in CGAS-CGAS_Score with NaN value
</div>

In [10]:
train[train['CGAS-CGAS_Score'] > 100]

,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,Stat_0,Stat_1,Stat_2,Stat_3,Stat_4,Stat_5,Stat_6,Stat_7,Stat_8,Stat_9,Stat_10,Stat_11,Stat_12,Stat_13,Stat_14,Stat_15,Stat_16,Stat_17,Stat_18,Stat_19,Stat_20,Stat_21,Stat_22,Stat_23,Stat_24,Stat_25,Stat_26,Stat_27,Stat_28,Stat_29,Stat_30,Stat_31,Stat_32,Stat_33,Stat_34,Stat_35,Stat_36,Stat_37,Stat_38,Stat_39,Stat_40,Stat_41,Stat_42,Stat_43,Stat_44,Stat_45,Stat_46,Stat_47,Stat_48,Stat_49,Stat_50,Stat_51,Stat_52,Stat_53,Stat_54,Stat_55,Stat_56,Stat_57,Stat_58,Stat_59,Stat_60,Stat_61,Stat_62,Stat_63,Stat_64,Stat_65,Stat_66,Stat_67,Stat_68,Stat_69,Stat_70,Stat_71,Stat_72,Stat_73,Stat_74,Stat_75,Stat_76,Stat_77,Stat_78,Stat_79,Stat_80,Stat_81,Stat_82,Stat_83,Stat_84,Stat_85,Stat_86,Stat_87,Stat_88,Stat_89,Stat_90,Stat_91,Stat_92,Stat_93,Stat_94,Stat_95,sii
2065,Fall,11,1,Winter,999.0,Fall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,Fall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,2.241,NaN,NaN,NaN,Fall,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
train.loc[train['CGAS-CGAS_Score'] == 999, 'CGAS-CGAS_Score'] = np.nan

<div style="font-size: 16px;">
Replacing zero stats in these physical features as NaN value
</div>

In [12]:
physical_cols = [
    'Physical-BMI', 'Physical-Height',
    'Physical-Weight', 'Physical-Waist_Circumference'
]
print((train[physical_cols] == 0).sum())

Physical-BMI                     7
Physical-Height                  0
Physical-Weight                 61
Physical-Waist_Circumference     0
dtype: int64


In [13]:
train[physical_cols] = train[physical_cols].replace(0, np.nan)
print((train[physical_cols] == 0).sum())

Physical-BMI                    0
Physical-Height                 0
Physical-Weight                 0
Physical-Waist_Circumference    0
dtype: int64


In [14]:
bp_cols = [
      'Physical-Diastolic_BP', 'Physical-Systolic_BP'
]
train[bp_cols] = train[bp_cols].replace(0, np.nan)
train.loc[train['Physical-Systolic_BP'] <= train['Physical-Diastolic_BP'], bp_cols] = np.nan

<div style="font-size: 16px;">
Replacing extreme outliers in BIA-related columns with NaN values.
</div>

In [15]:
bia_columns = [col for col in train.columns if 'BIA' in col]

cat_col = ['BIA-Season', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_Frame_num']
bia_columns = [col for col in bia_columns if col not in cat_col]
bia_columns

exclude_rows = []  # To store all indices to exclude

for col in bia_columns:
    Q1 = train[col].quantile(0.001)  # Lower quantile
    Q3 = train[col].quantile(0.999)  # Upper quantile

    # Replace with nan 
    train.loc[train[col] > Q3, col] = np.nan  # Replace above Q3 with NaN
    train.loc[train[col] < Q1, col] = np.nan  # Replace below Q1 with NaN

#  Dropping rows 
#     exclude_q3 = train[train[col] > Q3].index.tolist()
#     exclude_q1 = train[train[col] < Q1].index.tolist()
    
#     exclude_rows += exclude_q3 + exclude_q1

# exclude_rows = list(set(exclude_rows))

# train = train.drop(index=exclude_rows)

train

,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,Stat_0,Stat_1,Stat_2,Stat_3,Stat_4,Stat_5,Stat_6,Stat_7,Stat_8,Stat_9,Stat_10,Stat_11,Stat_12,Stat_13,Stat_14,Stat_15,Stat_16,Stat_17,Stat_18,Stat_19,Stat_20,Stat_21,Stat_22,Stat_23,Stat_24,Stat_25,Stat_26,Stat_27,Stat_28,Stat_29,Stat_30,Stat_31,Stat_32,Stat_33,Stat_34,Stat_35,Stat_36,Stat_37,Stat_38,Stat_39,Stat_40,Stat_41,Stat_42,Stat_43,Stat_44,Stat_45,Stat_46,Stat_47,Stat_48,Stat_49,Stat_50,Stat_51,Stat_52,Stat_53,Stat_54,Stat_55,Stat_56,Stat_57,Stat_58,Stat_59,Stat_60,Stat_61,Stat_62,Stat_63,Stat_64,Stat_65,Stat_66,Stat_67,Stat_68,Stat_69,Stat_70,Stat_71,Stat_72,Stat_73,Stat_74,Stat_75,Stat_76,Stat_77,Stat_78,Stat_79,Stat_80,Stat_81,Stat_82,Stat_83,Stat_84,Stat_85,Stat_86,Stat_87,Stat_88,Stat_89,Stat_90,Stat_91,Stat_92,Stat_93,Stat_94,Stat_95,sii
0,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,NaN,Fall,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,Fall,2.0,2.66855,16.8792,932.498,1492.00,8.25598,41.5862,13.8177,3.06143,9.21377,1.0,24.4349,8.89536,38.9177,19.5413,32.6909,NaN,NaN,NaN,NaN,NaN,Fall,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
1,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,NaN,Fall,3.0,0.0,NaN,NaN,NaN,NaN,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,Winter,2.0,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.21172,3.97085,1.0,21.0352,14.97400,39.4497,15.4107,27.0552,Fall,2.340,Fall,46.0,64.0,Summer,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,NaN,65.0,94.0,117.0,Fall,Fall,20.0,1.0,10.2,1.0,14.7,2.0,7.0,1.0,10.0,1.0,10.0,1.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,NaN,60.0,97.0,117.0,Summer,Summer,18.0,1.0,NaN,NaN,NaN,NaN,5.0,0.0,7.0,0.0,7.0,0.0,7.0,1.0,Summer,3.0,3.84191,18.2943,1131.430,1923.44,15.59250,62.7757,14.0740,4.22033,18.82430,2.0,30.4041,16.77900,58.9338,26.4798,45.9966,Winter,2.451,Summer,31.0,45.0,Winter,0.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,-0

# Data Preprocessing

<h2>Feature Engineering</h2>

<div style="font-size: 16px;">
Dropping features have importance = 0 when training Light model based on different seeds
</div>

In [16]:
# Dropping features have importance = 0 when training Light model based on different seeds

columns_drop = ['Stat_39', 'Stat_45', 'Stat_41', 'Stat_89', 'Stat_6', 'Stat_42', 
                 'Stat_7', 'Stat_10', 'Stat_9', 'Stat_11'
                , 'Stat_93', 'Stat_8'
               ]
train = train.drop(columns = columns_drop)
test = test.drop(columns = columns_drop)


In [17]:
# # Re-creating interaction features

# # Interaction between Physical-Height and Physical-Weight
# train["Height_Weight"] = train["Physical-Height"] * train["Physical-Weight"]

# # Interaction between Basic_Demos-Age and Internet_Use_Hours
# train["Age_Internet_Hours"] = train["Basic_Demos-Age"] * train["PreInt_EduHx-computerinternet_hoursday"]

# # Interaction between Physical-Height and Basic_Demos-Age
# train["Height_Age"] = train["Physical-Height"] * train["Basic_Demos-Age"]

# # Interaction between Physical-Weight and Fitness-Endurance-Time_Sec
# train["Weight_Endurance"] = train["Physical-Weight"] * train["Physical-Height"]

In [18]:
def feature_transformation(data):
    # data['Height_squared'] = data['Physical-Height'] ** 2
    # data['calculated_BMI'] = data['Physical-Weight'] / ((data['Physical-Height'] ** 2) + 1e-5)
    # data['BP_Ratio'] = data['Physical-Systolic_BP'] / (data['Physical-Diastolic_BP'] + 1e-5)
    # data['Fat_to_Lean_Ratio'] = data['BIA-BIA_FMI'] / (data['BIA-BIA_FFMI'] + 1e-5)
    # data['Fat_to_LST_Ratio'] = data['BIA-BIA_Fat'] / (data['BIA-BIA_LST'] + 1e-5)
    # data['Water_Balance'] = data['BIA-BIA_ICW'] / (data['BIA-BIA_ECW'] + 1e-5)
    # data['ICW_TBW_Ratio'] = data['BIA-BIA_ICW'] / data['BIA-BIA_TBW']
    # data['ECW_TBW_Ratio'] = data['BIA-BIA_ECW'] / data['BIA-BIA_TBW']
    # data['Metabolic_Efficiency'] = data['BIA-BIA_DEE'] / (data['BIA-BIA_BMR'] + 1e-5)
    # data['SMM_to_LDM_Ratio'] = data['BIA-BIA_SMM'] / (data['BIA-BIA_LDM'] + 1e-5)
    # data['Total_Body_Mass'] = data['BIA-BIA_FFM'] + data['BIA-BIA_Fat']
    # data['Activity_Sleep_Ratio'] = data['PAQ_C-PAQ_C_Total'] / (data['SDS-SDS_Total_Raw'] + 1e-5)
    # data['Strength_Efficiency'] = data['FGC-FGC_CU'] / (data['PAQ_C-PAQ_C_Total'] + 1e-5)
    # data['FGC_PAQ_C_Interaction'] = data['FGC-FGC_CU'] * data['PAQ_C-PAQ_C_Total']

    # col = [
    #     'Physical-Systolic_BP', 'Physical-Diastolic_BP', 'BIA-BIA_FMI', 'BIA-BIA_FFMI', 'BIA-BIA_Fat',
    #     'BIA-BIA_LST', 'BIA-BIA_ICW', 'BIA-BIA_ECW'
    # ]
    # data = data.drop(columns = col, axis = 1)

    # data['Age_HeartRate_Ratio'] = data['Physical-HeartRate'] / (data['Basic_Demos-Age'] + 1e-5)
    # data['Activity_SDS_Impact'] = data['PAQ_C-PAQ_C_Total'] / (data['SDS-SDS_Total_Raw'] + 1e-5)
    # data['Age_SDS_Interaction'] = data['Basic_Demos-Age'] * data['SDS-SDS_Total_Raw']
    # data['Age_Internet_Use'] = data['PreInt_EduHx-computerinternet_hoursday'] * data['Basic_Demos-Age']
    # data['Weight_Increase'] = data['Basic_Demos-Age'] * data['Physical-Weight']
    # data['Height_Increase'] = data['Basic_Demos-Age'] * data['Physical-Height']

    # data['BMI_difference'] = data['Physical-BMI'] - data['calculated_BMI']
    return data


In [19]:
#train = feature_transformation(train)
#test = feature_transformation(test)

<div style="font-size: 16px;">
Dropping out rows that contain NaN value for target column
</div>

In [20]:
# train = train[featuresCols]
train = train.dropna(subset='sii')
train

,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,Stat_0,Stat_1,Stat_2,Stat_3,Stat_4,Stat_5,Stat_12,Stat_13,Stat_14,Stat_15,Stat_16,Stat_17,Stat_18,Stat_19,Stat_20,Stat_21,Stat_22,Stat_23,Stat_24,Stat_25,Stat_26,Stat_27,Stat_28,Stat_29,Stat_30,Stat_31,Stat_32,Stat_33,Stat_34,Stat_35,Stat_36,Stat_37,Stat_38,Stat_40,Stat_43,Stat_44,Stat_46,Stat_47,Stat_48,Stat_49,Stat_50,Stat_51,Stat_52,Stat_53,Stat_54,Stat_55,Stat_56,Stat_57,Stat_58,Stat_59,Stat_60,Stat_61,Stat_62,Stat_63,Stat_64,Stat_65,Stat_66,Stat_67,Stat_68,Stat_69,Stat_70,Stat_71,Stat_72,Stat_73,Stat_74,Stat_75,Stat_76,Stat_77,Stat_78,Stat_79,Stat_80,Stat_81,Stat_82,Stat_83,Stat_84,Stat_85,Stat_86,Stat_87,Stat_88,Stat_90,Stat_91,Stat_92,Stat_94,Stat_95,sii
0,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,NaN,Fall,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,Fall,2.0,2.66855,16.8792,932.498,1492.00,8.25598,41.5862,13.8177,3.061430,9.21377,1.0,24.4349,8.89536,38.9177,19.5413,32.6909,NaN,NaN,NaN,NaN,NaN,Fall,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
1,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,NaN,Fall,3.0,0.0,NaN,NaN,NaN,NaN,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,Winter,2.0,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.211720,3.97085,1.0,21.0352,14.97400,39.4497,15.4107,27.0552,Fall,2.340,Fall,46.0,64.0,Summer,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,NaN,65.0,94.0,117.0,Fall,Fall,20.0,1.0,10.2,1.0,14.7,2.0,7.0,1.0,10.0,1.0,10.0,1.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,NaN,60.0,97.0,117.0,Summer,Summer,18.0,1.0,NaN,NaN,NaN,NaN,5.0,0.0,7.0,0.0,7.0,0.0,7.0,1.0,Summer,3.0,3.84191,18.2943,1131.430,1923.44,15.59250,62.7757,14.0740,4.220330,18.82430,2.0,30.4041,16.77900,58.9338,26.4798,45.9966,Winter,2.451,Summer,31.0,45.0,Winter,0.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,-0.316384,0.016009,-0.167890,0.047388,-10.580416,0.000000,42.296310,4053.579102,5.046215e+13,4.470182,3.0,53.201683,0.453665,0.502702,0.585710,0.106351,42.947170,0.000000,208.168976,112.404045,1.942842e+13,1.931421,0.0,14.244914,-1.746094,-2.905339,-1.048372,-89.833092,3824.000000,5

<div style="font-size: 18px;">
Here we perform encoding the categorical features
</div>

<div style="font-size: 16px;">
Fill the NaN values of categorical features as <strong>Missing</strong>
</div>

In [21]:
cat_c = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 'Fitness_Endurance-Season', 
          'FGC-Season', 'BIA-Season', 
         'PAQ_A-Season', 
         'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']
final_cat_columns = [col for col in cat_c if col in train.columns]

def update(df):
    for c in final_cat_columns: 
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')
    return df

train = update(train)
test = update(test)


<div style="font-size: 16px;">
Mapping to numerical values for the model to train
</div>

In [22]:
def create_mapping(column, dataset):
    unique_values = dataset[column].unique()
# enumerate adds index to an iterable. unique_values = ['A','B','C'] => enumerate returns [(0, 'A'), (1, 'B'), (2, 'C')]
    return {value: idx for idx, value in enumerate(unique_values)}

# Similar to Label Encoder
for col in final_cat_columns:
    all_values = pd.concat([train[col], test[col]]).unique()
    mapping = {value: idx for idx, value in enumerate(all_values)}

    train[col] = train[col].replace(mapping).astype(int)
    test[col] = test[col].replace(mapping).astype(int)

In [23]:
# test = test[train.columns.drop('sii')]  # Align test columns to training features

# Model Training and Evaluation

In [24]:
SEED = 42
n_splits = 5

In [25]:
# Low importance features across different seeds
low_imp_features = []

In [26]:
# High importance features across different seeds
high_imp_features = []

<h2>Supporting Functions</h2>
    <ul style="font-size: 16px;">
        <li><strong>quadratic_weighted_kappa</strong>: Calculates the QWK score.</li>
        <li><strong>threshold_Rounder</strong>: Rounds the <code>sii</code> from <code>PCIAT_Total</code> to correct labels.</li>
        <li><strong>evaluate_predictions</strong>: Combines the two functions above to round continuous predictions and return the negative QWK score for optimization using the minimize library.</li>
    </ul>

 <h2>TrainML Function</h2>
    <p style="font-size: 16px;">The <strong>TrainML</strong> function performs the following tasks:</p>
        <ul style="font-size: 16px;">
        <li>Implements stratified k-fold cross-validation to train and validate the given model.</li>
        <li>Trains a cloned instance of the input model on training data for each fold.</li>
        <li>Calculates and prints QWK scores for both training and validation sets.</li>
        <li>Generates predictions for the test dataset during each fold and averages them.</li>
        <li>Optimizes the thresholds for rounding predictions to maximize QWK using a threshold tuning function and the Nelder-Mead optimization method.</li>
    </ul>

In [27]:
%%time


#We need Confusion matrix, weight matrix and expected matrix
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')


def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))


# Optimization libraries like scipy.optimize.minimize work by minimizing a function. 
# Since QWK is a metric where higher is better, we negate it to allow the optimizer to "maximize" QWK indirectly.
def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

# TrainML, is a machine learning pipeline that performs model training, cross-validation, evaluation, 
# and test set prediction for a classification or regression task. 

def TrainML(model_class, test_data
            # , k_values=[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
           ):
    
    X = train.drop(['sii'], axis=1)
    y = train['sii']

    # best_kappa = -np.inf
    # best_k = None
    # best_model = None
    # best_submission = None

    # for n_splits in k_values:
    #     print(f"\nEvaluating for k = {n_splits} folds...")

    # Cross-validation, ensuring that the folds have the same distribution of target classes. 
    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    # Store QWK scores for training and validation sets.
    train_S = []
    test_S = []

    # predictions made by the model for the validation set during each fold of cross-validation.
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    # discrete predictions derived from oof_non_rounded using rounding logic
    oof_rounded = np.zeros(len(y), dtype=int) 
    # Stores the model's predictions for the test dataset during each fold of cross-validation
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        # train and test index are the index of this fold in the original training set
        # Data used for training in this fold
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        # Data used for validation in this fold
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]
        
        
        # Creates a new instance of an estimator with the same parameters as the original one 
        # but without any of the fitted data or state. Ensures each fold starts with a clean estimator.
        model = clone(model_class)
        model.fit(X_train, y_train)

        
        # Predict after fit to ensure model is trained correctly
        y_train_pred = model.predict(X_train)

        y_train_pred_rounded = y_train_pred.round(0).astype(int)
        
        y_val_pred = model.predict(X_val)

        # stores continuous data
        oof_non_rounded[test_idx] = y_val_pred
        # round the predicted value
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        # Calcute QWK for train and validation set
        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred_rounded)
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        # Stores QWK score each fold
        train_S.append(train_kappa)
        test_S.append(val_kappa)

        
        test_preds[:, fold] = model.predict(test_data)
        print(test_preds[:, fold])
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)


    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    #Minimize the kappa score, which find the best Threshold Rounder  
    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead') # Nelder-Mead | # Powell

    # If not converge, raise error
    assert KappaOPtimizer.success, "Optimization did not converge."

# rounded the oof_non_rounded which has a bunch of predicted continuous variables for the validation set at each fold
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

# Rounded the test predict variables with optimized ThresHold_Rounder
    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned
    })

    return submission,model

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 11 µs


In [28]:
%%time

# LightParams = {'num_leaves': 443, 'max_depth': 11, 'learning_rate': 0.03975148176099325, 
#                'feature_fraction': 0.7369224778898958, 'bagging_fraction': 0.7673760625886821, 
#            'bagging_freq': 2, 'lambda_l1': 4.329242178275905, 'lambda_l2': 8.559080320318334e-06, 
#                'min_data_in_leaf': 11
# }
LightParams = {
    'num_leaves': 484, 'max_depth': 11, 'learning_rate': 0.04533585929025977, 
    'feature_fraction': 0.8110477902071817, 'bagging_fraction': 0.7265461551046623, 'bagging_freq': 2, 
    'lambda_l1': 5.338437863405547, 'lambda_l2': 4.499492326361118e-06, 'min_data_in_leaf': 13
}
# train 0.8276 cv 0.4067 opt qwk 0.453

# LightParams = {
#     'num_leaves': 405, 'max_depth': 14, 'learning_rate': 0.04144286287843841,
#     'feature_fraction': 0.7449300074156723, 'bagging_fraction': 0.728943178243488, 'bagging_freq': 2, 
#     'lambda_l1': 5.47005971080022, 'lambda_l2': 8.207849169164218e-06, 'min_data_in_leaf': 11
# } train 0.8201 cv 0.3971 opt qwk 0.452

Light = lgb.LGBMRegressor(**LightParams,random_state=SEED, verbose=-1,n_estimators=200)

Submission,model = TrainML(Light,test)

Training Folds: 100%|██████████| 5/5 [00:14<00:00,  2.90s/it]

Mean Train QWK --> 0.8276
Mean Validation QWK ---> 0.4067


----> || Optimized QWK SCORE ::  0.453
CPU times: user 14.6 s, sys: 1.17 s, total: 15.7 s
Wall time: 14.8 s


# Feature Importances

In [29]:
feature_importances = pd.DataFrame({
    'Feature': model.booster_.feature_name(),
    'Importance': model.booster_.feature_importance(importance_type='gain')
}).sort_values(by='Importance', ascending=False)

feature_importances

,Feature,Importance
1,Basic_Demos-Age,1018.068766
52,PreInt_EduHx-computerinternet_hoursday,792.141328
49,SDS-SDS_Total_Raw,704.473046
7,Physical-Height,406.427823
8,Physical-Weight,267.751972
...,...,...
108,Stat_65,0.000000
101,Stat_58,0.000000
134,Stat_92,0.000000
96,Stat_53,0.000000


<div style="font-size: 16px;">
    Here i examine the high and low importance features accross different seeds, and perform feature engineering 
</div>

In [30]:
# high_importance_features = feature_importances['Feature'].head(10)
# high_importance_features = high_importance_features.tolist()

# high_imp_features.append(high_importance_features)

In [31]:
# from itertools import combinations

# # Function to compute distinct common strings
# def find_distinct_common_strings(array):
#     results = {}

#     n = len(array)  # Total number of lists in the array

#     # Convert all lists to sets for easier intersection
#     sets = [set(lst) for lst in array]

#     # Keep track of already identified strings
#     identified_strings = set()

#     # For each exclusion level (0 to n-1)
#     for exclude_count in range(n):
#         current_level_strings = set()
        
#         # Generate combinations of lists with `exclude_count` excluded
#         for excluded_indices in combinations(range(n), exclude_count):
#             # Include only the indices not in excluded_indices
#             included_sets = [sets[i] for i in range(n) if i not in excluded_indices]
#             # Find intersection of included sets
#             if included_sets:
#                 intersection = set.intersection(*included_sets)
#                 # Exclude strings already identified in previous levels
#                 intersection -= identified_strings
#                 current_level_strings.update(intersection)
        
#         # Store results for this exclusion level
#         level_name = f"length-{n - exclude_count}"
#         results[level_name] = current_level_strings

#         # Update identified strings to include the current level's strings
#         identified_strings.update(current_level_strings)

#     return results

# distinct_common_strings = find_distinct_common_strings(high_imp_features)

# for key, value in distinct_common_strings.items():
#     print(f"{key}: {value}")


In [32]:
Submission.to_csv('submission.csv', index=False)
print(Submission['sii'].value_counts())

sii
0    10
2     5
1     5
Name: count, dtype: int64
